In [26]:
import os
import requests
import fitz 
from tqdm.auto import tqdm
import random
import pandas as pd
from spacy.lang.en import English
# Get PDF document path


In [15]:
pdf_path = "resources/spring-framework.pdf"

# Download PDF
if not os.path.exists(pdf_path):
    print("[INFO] File doesn't exist, downloading...")
    url = "https://docs.spring.io/spring-framework/docs/6.0.0/reference/pdf/spring-framework.pdf"
    filename = pdf_path
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, "wb") as file:
            file.write(response.content) 
        print(f"[INFO] The file has been download and saved as {filename}")
    else:
        print(f"[INFO] Failed to download the file. Status code: {reponse.status_code}")
else:
    print(f"File {pdf_path} exists.")



File resources/spring-framework.pdf exists.


In [16]:
def text_formatter(text: str) -> str: 
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = [] 
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_setence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4, # 1 token = ~4 characters
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': 0,
  'page_char_count': 44,
  'page_word_count': 5,
  'page_setence_count_raw': 1,
  'page_token_count': 11.0,
  'text': 'Spring Framework Documentation Version 6.0.0'},
 {'page_number': 1,
  'page_char_count': 3152,
  'page_word_count': 494,
  'page_setence_count_raw': 31,
  'page_token_count': 788.0,
  'text': 'Chapter 1. Spring Framework Overview Spring makes it easy to create Java enterprise applications. It provides everything you need to embrace the Java language in an enterprise environment, with support for Groovy and Kotlin as alternative languages on the JVM, and with the flexibility to create many kinds of architectures depending on an application’s needs. As of Spring Framework 5.1, Spring requires JDK 8+ (Java SE 8+) and provides out-of-the-box support for JDK 11 LTS. Java SE 8 update 60 is suggested as the minimum patch release for Java 8, but it is generally recommended to use a recent patch release. Spring supports a wide range of application scenarios.

In [17]:
random.sample(pages_and_texts, k=3)

[{'page_number': 439,
  'page_char_count': 407,
  'page_word_count': 66,
  'page_setence_count_raw': 2,
  'page_token_count': 101.75,
  'text': 'Java @Configuration(proxyBeanMethods = false) public class DataSourceConfiguration { \xa0   @Bean \xa0   public SimpleDataSource dataSource() { \xa0       return new SimpleDataSource(); \xa0   } } Since there isn’t any particular condition on this class, dataSourceConfiguration and dataSource are identified as candidates. The AOT engine will convert the configuration class above to code similar to the following: 439'},
 {'page_number': 278,
  'page_char_count': 1658,
  'page_word_count': 233,
  'page_setence_count_raw': 8,
  'page_token_count': 414.5,
  'text': 'Java ExpressionParser parser = new SpelExpressionParser(); Expression exp = parser.parseExpression("new String(\'hello world\').toUpperCase()"); ① String message = exp.getValue(String.class); ① Construct a new String from the literal and make it be upper case. Kotlin val parser = SpelE

In [23]:
df = pd.DataFrame(pages_and_texts)
df.set_index("page_number")
df.head()

,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,text
0,0,44,5,1,11.00,Spring Framework Documentation Version 6.0.0
1,1,3152,494,31,788.00,Chapter 1. Spring Framework Overview Spring ma...
2,2,2621,418,25,655.25,specifications from the traditional EE umbrell...
3,3,2207,366,23,551.75,• Maintain strong backward compatibility. Spri...
4,4,2720,386,23,680.00,Chapter 2. Core Technologies This part of the ...


In [24]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count
count,1429.00,1429.00,1429.00,1429.00,1429.00
mean,714.00,1725.97,268.72,10.10,431.49
std,412.66,595.52,92.28,6.43,148.88
min,0.00,44.00,5.00,1.00,11.00
25%,357.00,1304.00,202.00,5.00,326.00
50%,714.00,1677.00,259.00,9.00,419.25
75%,1071.00,2090.00,325.00,14.00,522.50
max,1428.00,3803.00,645.00,38.00,950.75


In [29]:
#Spacy pipeline creator

nlp = English()
nlp.add_pipe("sentencizer")


In [ ]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    item["page_sentence_count_spacy"] = len(item["sentences"])

random.sample(pages_and_texts, k=1)